Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/deployment/production-deploy-to-aks/production-deploy-to-aks.png)

# Deploying a web service to Azure Kubernetes Service (AKS)
This notebook shows the steps for deploying a service: registering a model, creating an image, provisioning a cluster (one time action), and deploying a service to it. 
We then test and delete the service, image and model.

In [1]:
from azureml.core import Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.model import Model

In [2]:
import azureml.core
print(azureml.core.VERSION)

1.34.0


# Get workspace
Load existing workspace from the config file info.

In [3]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

mlrgeast
mlrg
eastus
932c3e14-d0cf-4e41-9998-bdebb9bfa1cf


# Register the model
Register an existing trained model, add descirption and tags.

In [4]:
#Register the model
from azureml.core.model import Model

model_name = "Youjin-test"
'''model = Model.register( model_path='./',
                        model_name=modelname,
                        tags={"data": "mosquito", "model": "regression"},
                        description="predict mosquito status",
                        workspace=ws)'''


model = Model(ws, model_name,version=1)
print(model.name, model.description, model.version)
#print(str(Model.get_model_path('myModelYoujin', 1, ws)))

Youjin-test None 1


In [5]:
print(Model.get_model_path(model_name,1,ws))

azureml-models/Youjin-test/1/Youjin_test_model.pkl


# Create the Environment
Create an environment that the model will be deployed with

In [6]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies 
'''
conda_deps = CondaDependencies.create(conda_packages=['numpy','tensorflow==1.12.0','scipy'], pip_packages=['azureml-defaults', 'inference-schema'])
myenv = Environment(name='myExp')
myenv.python.conda_dependencies = conda_deps'''

from azureml.core import Environment

env = Environment.from_conda_specification(name = 'keras-2.3.1', file_path = './conda_dependencies.yml')

'''# Specify a GPU base image
env.docker.enabled = True
env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.0-cudnn7-ubuntu18.04'''

env.register(workspace = ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210806.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "keras-2.3.1",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "conda-forge"
            ],
            "de

In [7]:
'''from azureml.core import Environment

keras_env = Environment.from_conda_specification(name = 'keras-2.3.1', file_path = './conda_dependencies.yml')

myenv = Environment(name='myExp')
myenv.python.conda_dependencies = keras_env'''

"from azureml.core import Environment\n\nkeras_env = Environment.from_conda_specification(name = 'keras-2.3.1', file_path = './conda_dependencies.yml')\n\nmyenv = Environment(name='myExp')\nmyenv.python.conda_dependencies = keras_env"

#### Use a custom Docker image

You can also specify a custom Docker image to be used as base image if you don't want to use the default base image provided by Azure ML. Please make sure the custom Docker image has Ubuntu >= 16.04, Conda >= 4.5.\* and Python(3.5.\* or 3.6.\*).

Only supported with `python` runtime.
```python
# use an image available in public Container Registry without authentication
myenv.docker.base_image = "mcr.microsoft.com/azureml/o16n-sample-user-base/ubuntu-miniconda"

# or, use an image available in a private Container Registry
myenv.docker.base_image = "myregistry.azurecr.io/mycustomimage:1.0"
myenv.docker.base_image_registry.address = "myregistry.azurecr.io"
myenv.docker.base_image_registry.username = "username"
myenv.docker.base_image_registry.password = "password"
```

# Write the Entry Script
Write the script that will be used to predict on your model

In [35]:
%%writefile score.py
import os
import pickle
import json
import numpy
import joblib
import tensorflow as tf
from tensorflow import keras

def init():
    global model
    #model_path=os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'Youjin_test')
    model = joblib.load('azureml-models/Youjin-test/1/Youjin_test_model.pkl')
    history = model.fit(x_train, y_train, epochs=eph_size, batch_size=4,validation_data=(x_val,y_val))
    score = model.evaluate(x_test, y_test)
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    
    #model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'myModelYoujin')
    # deserialize the model file back into a sklearn model
   #model_path = os.path.join('./model', 'myModelYoujin')
   # model = joblib.load(model_path)

# note you can pass in multiple rows for scoring
def run(raw_data):
    try:
        input_json = {'data': [[20.5, 18.1, 23.1, 2.3, 93.3, 82.0],
                        [23.7, 21.9, 26.3, 1.6, 82.6, 79.3]]}
    # create a string that can be put in the body of the request
        data = json.dumps(input_json)
        data = numpy.array(data)
        result = model.predict(data)
        # you can return any data type as long as it is JSON-serializable
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

Overwriting score.py


# Create the InferenceConfig
Create the inference config that will be used when deploying the model

In [22]:
from azureml.core.model import InferenceConfig

inf_config = InferenceConfig(entry_script='score.py', environment=env)

In [10]:
'''from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice


service_name = 'youjin-deploy-test'

inference_config = InferenceConfig(entry_script='score.py', environment=myenv)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)'''

"from azureml.core.model import InferenceConfig\nfrom azureml.core.webservice import AciWebservice\n\n\nservice_name = 'youjin-deploy-test'\n\ninference_config = InferenceConfig(entry_script='score.py', environment=myenv)\naci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)\n\nservice = Model.deploy(workspace=ws,\n                       name=service_name,\n                       models=[model],\n                       inference_config=inference_config,\n                       deployment_config=aci_config,\n                       overwrite=True)\nservice.wait_for_deployment(show_output=True)"

# Model Profiling

Profile your model to understand how much CPU and memory the service, created as a result of its deployment, will need. Profiling returns information such as CPU usage, memory usage, and response latency. It also provides a CPU and memory recommendation based on the resource usage. You can profile your model (or more precisely the service built based on your model) on any CPU and/or memory combination where 0.1 <= CPU <= 3.5 and 0.1GB <= memory <= 15GB. If you do not provide a CPU and/or memory requirement, we will test it on the default configuration of 3.5 CPU and 15GB memory.

In order to profile your model you will need:
- a registered model
- an entry script
- an inference configuration
- a single column tabular dataset, where each row contains a string representing sample request data sent to the service.

Please, note that profiling is a long running operation and can take up to 25 minutes depending on the size of the dataset.

At this point we only support profiling of services that expect their request data to be a string, for example: string serialized json, text, string serialized image, etc. The content of each row of the dataset (string) will be put into the body of the HTTP request and sent to the service encapsulating the model for scoring.

Below is an example of how you can construct an input dataset to profile a service which expects its incoming requests to contain serialized json. In this case we created a dataset based one hundred instances of the same request data. In real world scenarios however, we suggest that you use larger datasets with various inputs, especially if your model resource usage/behavior is input dependent.

You may want to register datasets using the register() method to your workspace so they can be shared with others, reused and referred to by name in your script.
You can try get the dataset first to see if it's already registered.

In [11]:
'''import json
from azureml.core import Datastore
from azureml.core.dataset import Dataset
from azureml.data import dataset_type_definitions

dataset_name='sample_request_data'

dataset_registered = False
try:
    sample_request_data = Dataset.get_by_name(workspace = ws, name = dataset_name)
    dataset_registered = True
except:
    print("The dataset {} is not registered in workspace yet.".format(dataset_name))

if not dataset_registered:
    input_json = {'data': [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                        [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]]}
    # create a string that can be put in the body of the request
    serialized_input_json = json.dumps(input_json)
    dataset_content = []
    for i in range(100):
        dataset_content.append(serialized_input_json)
    sample_request_data = '\n'.join(dataset_content)
    file_name = "{}.txt".format(dataset_name)
    f = open(file_name, 'w')
    f.write(sample_request_data)
    f.close()

    # upload the txt file created above to the Datastore and create a dataset from it
    data_store = Datastore.get_default(ws)
    data_store.upload_files(['./' + file_name], target_path='sample_request_data')
    datastore_path = [(data_store, 'sample_request_data' +'/' + file_name)]
    sample_request_data = Dataset.Tabular.from_delimited_files(
        datastore_path,
        separator='\n',
        infer_column_types=True,
        header=dataset_type_definitions.PromoteHeadersBehavior.NO_HEADERS)
    sample_request_data = sample_request_data.register(workspace=ws,
                                                    name=dataset_name,
                                                    create_new_version=True)'''

'import json\nfrom azureml.core import Datastore\nfrom azureml.core.dataset import Dataset\nfrom azureml.data import dataset_type_definitions\n\ndataset_name=\'sample_request_data\'\n\ndataset_registered = False\ntry:\n    sample_request_data = Dataset.get_by_name(workspace = ws, name = dataset_name)\n    dataset_registered = True\nexcept:\n    print("The dataset {} is not registered in workspace yet.".format(dataset_name))\n\nif not dataset_registered:\n    input_json = {\'data\': [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],\n                        [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]]}\n    # create a string that can be put in the body of the request\n    serialized_input_json = json.dumps(input_json)\n    dataset_content = []\n    for i in range(100):\n        dataset_content.append(serialized_input_json)\n    sample_request_data = \'\n\'.join(dataset_content)\n    file_name = "{}.txt".format(dataset_name)\n    f = open(file_name, \'w\')\n    f.write(sample_request_data)\n    f.close()\n\n    # up

Now that we have an input dataset we are ready to go ahead with profiling. In this case we are testing the previously introduced sklearn regression model on 1 CPU and 0.5 GB memory. The memory usage and recommendation presented in the result is measured in Gigabytes. The CPU usage and recommendation is measured in CPU cores.

In [12]:
'''from datetime import datetime
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import Model, InferenceConfig


environment = Environment('my-sklearn-environment')
environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults',
    'inference-schema[numpy-support]',
    'joblib',
    'numpy',
    'scikit-learn==0.19.1',
    'scipy'
])
inference_config = InferenceConfig(entry_script='score.py', environment=environment)
# if cpu and memory_in_gb parameters are not provided
# the model will be profiled on default configuration of
# 3.5CPU and 15GB memory
profile = Model.profile(ws,
            'sklearn-%s' % datetime.now().strftime('%m%d%Y-%H%M%S'),
            [model],
            inference_config,
            input_dataset=sample_request_data,
            cpu=1.0,
            memory_in_gb=0.5)

# profiling is a long running operation and may take up to 25 min
profile.wait_for_completion(True)
details = profile.get_details()'''

"from datetime import datetime\nfrom azureml.core import Environment\nfrom azureml.core.conda_dependencies import CondaDependencies\nfrom azureml.core.model import Model, InferenceConfig\n\n\nenvironment = Environment('my-sklearn-environment')\nenvironment.python.conda_dependencies = CondaDependencies.create(pip_packages=[\n    'azureml-defaults',\n    'inference-schema[numpy-support]',\n    'joblib',\n    'numpy',\n    'scikit-learn==0.19.1',\n    'scipy'\n])\ninference_config = InferenceConfig(entry_script='score.py', environment=environment)\n# if cpu and memory_in_gb parameters are not provided\n# the model will be profiled on default configuration of\n# 3.5CPU and 15GB memory\nprofile = Model.profile(ws,\n            'sklearn-%s' % datetime.now().strftime('%m%d%Y-%H%M%S'),\n            [model],\n            inference_config,\n            input_dataset=sample_request_data,\n            cpu=1.0,\n            memory_in_gb=0.5)\n\n# profiling is a long running operation and may take u

# Provision the AKS Cluster
This is a one time setup. You can reuse this cluster for multiple deployments after it has been created. If you delete the cluster or the resource group that contains it, then you would have to recreate it.

> Note that if you have an AzureML Data Scientist role, you will not have permission to create compute resources. Talk to your workspace or IT admin to create the compute targets described in this section, if they do not already exist.

In [24]:
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your AKS cluster
aks_name = 'my-aks-9' 

# Verify that cluster does not exist already
try:
    aks_target = ComputeTarget(workspace=ws, name=aks_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # Use the default configuration (can also provide parameters to customize)
    prov_config = AksCompute.provisioning_configuration(vm_size='STANDARD_D11_V2', location='East US')

    # Create the cluster
    aks_target = ComputeTarget.create(workspace = ws, 
                                    name = aks_name, 
                                    provisioning_configuration = prov_config)

if aks_target.get_status() != "Succeeded":
    aks_target.wait_for_completion(show_output=True)

Found existing cluster, use it.


# Create AKS Cluster in an existing virtual network (optional)
See code snippet below. Check the documentation [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-enable-virtual-network#use-azure-kubernetes-service) for more details.

In [14]:
# from azureml.core.compute import ComputeTarget, AksCompute

# # Create the compute configuration and set virtual network information
# config = AksCompute.provisioning_configuration(location="eastus2")
# config.vnet_resourcegroup_name = "mygroup"
# config.vnet_name = "mynetwork"
# config.subnet_name = "default"
# config.service_cidr = "10.0.0.0/16"
# config.dns_service_ip = "10.0.0.10"
# config.docker_bridge_cidr = "172.17.0.1/16"

# # Create the compute target
# aks_target = ComputeTarget.create(workspace = ws,
#                                   name = "myaks",
#                                   provisioning_configuration = config)

# Enable SSL on the AKS Cluster (optional)
See code snippet below. Check the documentation [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-secure-web-service) for more details

In [15]:
# provisioning_config = AksCompute.provisioning_configuration(ssl_cert_pem_file="cert.pem", ssl_key_pem_file="key.pem", ssl_cname="www.contoso.com")

In [16]:
'''%%time
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)'''

'%%time\naks_target.wait_for_completion(show_output = True)\nprint(aks_target.provisioning_state)\nprint(aks_target.provisioning_errors)'

## Optional step: Attach existing AKS cluster

If you have existing AKS cluster in your Azure subscription, you can attach it to the Workspace.

In [17]:
'''# # Use the default configuration (can also provide parameters to customize)
resourceid = '/subscriptions/932c3e14-d0cf-4e41-9998-bdebb9bfa1cf/resourcegroups/MLRG/providers/Microsoft.ContainerService/managedClusters/youjin-aks3c41624805d'

create_name='Youjin-aks-3'
# # Create the cluster
attach_config = AksCompute.attach_configuration(resource_id=resourceid)
aks_target = ComputeTarget.attach(workspace=ws, name=create_name, attach_configuration=attach_config)
# # Wait for the operation to complete
aks_target.wait_for_completion(True)'''

"# # Use the default configuration (can also provide parameters to customize)\nresourceid = '/subscriptions/932c3e14-d0cf-4e41-9998-bdebb9bfa1cf/resourcegroups/MLRG/providers/Microsoft.ContainerService/managedClusters/youjin-aks3c41624805d'\n\ncreate_name='Youjin-aks-3'\n# # Create the cluster\nattach_config = AksCompute.attach_configuration(resource_id=resourceid)\naks_target = ComputeTarget.attach(workspace=ws, name=create_name, attach_configuration=attach_config)\n# # Wait for the operation to complete\naks_target.wait_for_completion(True)"

# Deploy web service to AKS

In [25]:
# Set the web service configuration (using default here)
#aks_config = AksWebservice.deploy_configuration()

# # Enable token auth and disable (key) auth on the webservice
aks_config = AksWebservice.deploy_configuration(token_auth_enabled=True, auth_enabled=False)


In [36]:
%%time
aks_service_name ='aks-service-12'

aks_service = Model.deploy(workspace=ws,
                           name=aks_service_name,
                           models=[model],
                           inference_config=inf_config,
                           deployment_config=aks_config,
                           deployment_target=aks_target)

aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-10-30 08:40:06+00:00 Creating Container Registry if not exists.
2021-10-30 08:40:07+00:00 Registering the environment.
2021-10-30 08:40:08+00:00 Use the existing image.
2021-10-30 08:40:09+00:00 Creating resources in AKS.
2021-10-30 08:40:10+00:00 Submitting deployment to compute.
2021-10-30 08:40:10+00:00 Checking the status of deployment aks-service-12.
Failed


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: cf88c209-0f6e-4b1f-ad3e-acc12217afbe
More information can be found using '.get_logs()'
Error:
{
  "code": "KubernetesDeploymentFailed",
  "statusCode": 400,
  "message": "Kubernetes Deployment failed",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, AttributeError: 'NoneType' object has no attribute 'fit', please run print(service.get_logs()) to get details."
    },
    {
      "code": "DeploymentFailed",
      "message": "Your container endpoint is not available. Please follow the steps to debug:
	1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. Please refer to https://aka.ms/debugimage#dockerlog for more information.
	2. You can also interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machi

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: cf88c209-0f6e-4b1f-ad3e-acc12217afbe
More information can be found using '.get_logs()'
Error:
{
  "code": "KubernetesDeploymentFailed",
  "statusCode": 400,
  "message": "Kubernetes Deployment failed",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, AttributeError: 'NoneType' object has no attribute 'fit', please run print(service.get_logs()) to get details."
    },
    {
      "code": "DeploymentFailed",
      "message": "Your container endpoint is not available. Please follow the steps to debug:
	1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. Please refer to https://aka.ms/debugimage#dockerlog for more information.
	2. You can also interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/

In [34]:
print(aks_service.get_logs())

/bin/bash: /azureml-envs/azureml_a8a0845f249cc373e9e7168f4f641be5/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_a8a0845f249cc373e9e7168f4f641be5/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_a8a0845f249cc373e9e7168f4f641be5/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_a8a0845f249cc373e9e7168f4f641be5/lib/libtinfo.so.5: no version information available (required by /bin/bash)
bash: /azureml-envs/azureml_a8a0845f249cc373e9e7168f4f641be5/lib/libtinfo.so.5: no version information available (required by bash)
2021-10-30T08:36:32,458745741+00:00 - gunicorn/run 
2021-10-30T08:36:32,456910632+00:00 - rsyslog/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-10-30T08:36:32,467969489+00:00 - iot-server/run 
2021-10-30T08:36:32,481813860+00:00 - nginx/run 
EdgeHubConnectionString 

# Test the web service using run method
We test the web sevice by passing data.
Run() method retrieves API keys behind the scenes to make sure that call is authenticated.

In [15]:
%%time
import json

test_sample = json.dumps({'data': [
    [20.5, 18.1, 23.1, 2.3, 93.3, 82.0], 
    [23.7, 21.9, 26.3, 1.6, 82.6, 79.3]
]})
test_sample = bytes(test_sample,encoding = 'utf8')

prediction = aks_service.run(input_data = test_sample)
print(prediction)

'NoneType' object has no attribute 'predict'
CPU times: user 12.7 ms, sys: 8 µs, total: 12.7 ms
Wall time: 233 ms


In [16]:
token, refresh_by = aks_service.get_token()
print(token)

eyJhbGciOiJSUzI1NiIsImtpZCI6IkE3MDRDNkIzMTRDOTI3RkM2MTlGNTE2OTFCODBEQkQ2NzU2QUEyODciLCJ0eXAiOiJKV1QifQ.eyJjYW5SZWZyZXNoIjoiRmFsc2UiLCJ3b3Jrc3BhY2VJZCI6ImIyOTlkZDFhLTcyMGUtNGYzNC05NjRjLTcxODhmMmY5NDM1ZSIsInRpZCI6IjE5NDExYjNhLWI3Y2EtNDhkOS1hMWUyLTMxNjk0ZDBjY2I2NSIsIm9pZCI6IjA2OGJiZjA0LTAzZjctNGQ3OS1hZDgwLTY3NzYwN2JjNmNiNyIsImFjdGlvbnMiOiJbXCJNaWNyb3NvZnQuTWFjaGluZUxlYXJuaW5nU2VydmljZXMvd29ya3NwYWNlcy9yZWFkXCIsXCJNaWNyb3NvZnQuTWFjaGluZUxlYXJuaW5nU2VydmljZXMvd29ya3NwYWNlcy9zZXJ2aWNlcy9ha3Mvc2NvcmUvYWN0aW9uXCIsXCJNaWNyb3NvZnQuTWFjaGluZUxlYXJuaW5nU2VydmljZXMvd29ya3NwYWNlcy9vbmxpbmVFbmRwb2ludHMvc2NvcmUvYWN0aW9uXCJdIiwic2VydmljZUlkIjoiYWtzLXNlcnZpY2UtOCIsImV4cCI6MTYzNTY2NTY0MSwiaXNzIjoiYXp1cmVtbCIsImF1ZCI6ImF6dXJlbWwifQ.fWe4XN6IHQKclLPOr4AW1CKtwZ6MOVLzGTWKFPA86SVgOwJ5dIFfLh_iAzlM1jy5WlYWxtcJQiFnv688T11VqH8ULXuRx8ry-6ByclpA5L7nRuBnGDFHfC04vwR12uQNi1oR2fcKGnIE31TNavusrzkSB-XFrcPNCEDolRI0eAjXJhJonXA1aqI7-1y59aRrnfFP4kNN-ZMxT26ZFAP1AzQrHkcpL7eEJ51nHyebu_SCel1scYUTSOzNAb64aFxynq1mLIK-n7fTS4redyKzbE

In [18]:
primary, secondary = aks_service.get_keys()
print(primary)

ERROR:azureml.core.webservice.webservice:Received bad response from Model Management Service:
Response Code: 400
Headers: {'Date': 'Sat, 30 Oct 2021 07:34:22 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'x-ms-client-request-id': '051421d8-7be7-4c25-9824-2682a16b5214', 'x-ms-client-session-id': '93693392-1c7f-413c-919d-247528741d1d', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload', 'X-Content-Type-Options': 'nosniff', 'x-request-time': '0.073'}
Content: b'{"code":"BadRequest","statusCode":400,"message":"The request is invalid.","details":[{"code":"AuthDisabled","message":"Authentication is disabled (authEnabled set to false). Enable service authentication to list/regenerate keys. Subscription: 932c3e14-d0cf-4e41-9998-bdebb9bfa1cf, ResourceGroup: mlrg, Workspace: mlrgeast, ACR: /subscriptions/932c3e14-d0cf-4e41-9998-bdebb9bfa1cf/resourcegro

WebserviceException: WebserviceException:
	Message: Received bad response from Model Management Service:
Response Code: 400
Headers: {'Date': 'Sat, 30 Oct 2021 07:34:22 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'x-ms-client-request-id': '051421d8-7be7-4c25-9824-2682a16b5214', 'x-ms-client-session-id': '93693392-1c7f-413c-919d-247528741d1d', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload', 'X-Content-Type-Options': 'nosniff', 'x-request-time': '0.073'}
Content: b'{"code":"BadRequest","statusCode":400,"message":"The request is invalid.","details":[{"code":"AuthDisabled","message":"Authentication is disabled (authEnabled set to false). Enable service authentication to list/regenerate keys. Subscription: 932c3e14-d0cf-4e41-9998-bdebb9bfa1cf, ResourceGroup: mlrg, Workspace: mlrgeast, ACR: /subscriptions/932c3e14-d0cf-4e41-9998-bdebb9bfa1cf/

# Test the web service using raw HTTP request (optional)
Alternatively you can construct a raw HTTP request and send it to the service. In this case you need to explicitly pass the HTTP header. This process is shown in the next 2 cells.

In [19]:
# # if (key) auth is enabled, retrieve the API keys. AML generates two keys.
#key1, Key2 = aks_service.get_keys()
#print(key1)

# # if token auth is enabled, retrieve the token.
access_token, refresh_after = aks_service.get_token()

In [20]:
# construct raw HTTP request and send to the service
#%%time

import requests

import json

test_sample = json.dumps({'data': [
     [20.5, 18.1, 23.1, 2.3, 93.3, 82.0], 
    [23.7, 21.9, 26.3, 1.6, 82.6, 79.3]
]})
test_sample = bytes(test_sample,encoding = 'utf8')

# # If (key) auth is enabled, don't forget to add key to the HTTP header.
# headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + key1}

# # If token auth is enabled, don't forget to add token to the HTTP header.
headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + access_token}

resp = requests.post(aks_service.scoring_uri, test_sample, headers=headers)


print("prediction:", resp.text)

prediction: "'NoneType' object has no attribute 'predict'"


# Clean up
Delete the service, image and model.

In [ ]:
%%time
aks_service.delete()
model.delete()